In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim as gs
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')
import fasttext

In [ ]:
train_dataset = pd.read_csv('../train_data/train_data_1_1', delimiter=',')
validation_dataset = pd.read_csv('../train_data/validation_data_1_1', delimiter=',')

In [ ]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [ ]:
MAX_PAD_TITLE = 55

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD_TITLE - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD_TITLE:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD_TITLE]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
train_article_title = []
train_article_main_passage = []
train_article_keywords = []

train_table_title = []
train_table_main_passage = []
train_table_keywords = []

train_label = []

for i,row in tqdm(train_dataset.iterrows()):
    
    train_article_title.append(create_embedding(row['article_page_title']))
    train_table_title.append(create_embedding(row['table_page_title']))    
    
    train_article_main_passage.append(create_embedding(row['article_meta_description']))
    train_table_main_passage.append(create_embedding(row['table_page_summary']))
    
    train_article_keywords.append(create_embedding(row['article_keywords']))
    train_table_keywords.append(create_embedding(row['table_page_keywords']))  
    
    train_label.append(row['label'])

In [ ]:
train_article_title = np.array(train_article_title, dtype='float16')
train_table_title = np.array(train_table_title, dtype='float16')

train_article_main_passage = np.array(train_article_main_passage, dtype='float16')
train_table_main_passage = np.array(train_table_main_passage, dtype='float16')

train_article_keywords = np.array(train_article_keywords, dtype='float16')
train_table_keywords = np.array(train_table_keywords, dtype='float16')

train_label = np.array(train_label)

In [ ]:
train_article_main_passage.shape

In [ ]:
validation_article_title = []
validation_article_main_passage = []
validation_article_keywords = []

validation_table_title = []
validation_table_main_passage = []
validation_table_keywords = []

validation_label = []

for i, row in tqdm(validation_dataset.iterrows()):
    
    validation_article_title.append(create_embedding(row['article_page_title']))
    validation_table_title.append(create_embedding(row['table_page_title']))  
    
    validation_article_main_passage.append(create_embedding(row['article_meta_description']))
    validation_table_main_passage.append(create_embedding(row['table_page_summary']))
    
    validation_article_keywords.append(create_embedding(row['article_keywords']))
    validation_table_keywords.append(create_embedding(row['table_page_keywords'])) 
    
    validation_label.append(row['label'])

In [ ]:
validation_article_title = np.array(validation_article_title)
validation_article_main_passage = np.array(validation_article_main_passage)

validation_table_title = np.array(validation_table_title)
validation_table_main_passage = np.array(validation_table_main_passage)

validation_article_keywords = np.array(validation_article_keywords)
validation_table_keywords = np.array(validation_table_keywords)

validation_label = np.array(validation_label)

In [ ]:
validation_article_keywords.shape

In [ ]:
def attention_method(query, key, value):
    
    scores = tf.matmul(query, key, transpose_b=True)
    
    distribution = tf.nn.softmax(scores)
    
    attention_matrix = tf.matmul(distribution, value)
    
    return attention_matrix

In [ ]:
def attention_model():
    
    # inputs
    article_title = tf.keras.Input(shape=(55,50), dtype='float32')
    article_main_passage = tf.keras.Input(shape=(55,50), dtype='float32')
    article_keywords = tf.keras.Input(shape=(55,50), dtype='float32')
    
    table_title = tf.keras.Input(shape=(55,50), dtype='float32')
    table_main_passage = tf.keras.Input(shape=(55,50), dtype='float32')
    table_keywords = tf.keras.Input(shape=(55,50), dtype='float32')
    
    # article and tables signal representations
    rep_article_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_title)
    rep_article_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_main_passage)
    rep_article_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_keywords)
    
    rep_table_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_title)
    rep_table_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_main_passage)
    rep_table_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_keywords)
    
    
    #level 1 first signal of article
    #attention article title and table title
    att_A_title_T_title = attention_method(rep_article_title,rep_table_title,rep_article_title)
    rep_att_A_title_T_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_title_T_title)
    vector_A_title_T_title = tf.keras.layers.Flatten()(rep_att_A_title_T_title)
    
    #attention article title and table text
    att_A_title_T_text = attention_method(rep_article_title,rep_table_text,rep_article_title)
    rep_att_A_title_T_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_title_T_text)
    vector_A_title_T_text = tf.keras.layers.Flatten()(rep_att_A_title_T_text)
    
    #attention article title and table keywords
    att_A_title_T_keywords = attention_method(rep_article_title,rep_table_keywords,rep_article_title)
    rep_att_A_title_T_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_title_T_keywords)
    vector_A_title_T_keywords = tf.keras.layers.Flatten()(rep_att_A_title_T_keywords)
    
    
    
    #level 2 second signal of article
    #attention article text and table title
    att_A_text_T_title = attention_method(rep_article_text,rep_table_title,rep_article_text)
    rep_att_A_text_T_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_text_T_title)
    vector_A_text_T_title = tf.keras.layers.Flatten()(rep_att_A_text_T_title)
    
    #attention article text and table text
    att_A_text_T_text = attention_method(rep_article_text,rep_table_text,rep_article_text)
    rep_att_A_text_T_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_text_T_text)
    vector_A_text_T_text = tf.keras.layers.Flatten()(rep_att_A_text_T_text)
    
    #attention article text and table keywords
    att_A_text_T_keywords = attention_method(rep_article_text,rep_table_keywords,rep_article_text)
    rep_att_A_text_T_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_text_T_keywords)
    vector_A_text_T_keywords = tf.keras.layers.Flatten()(rep_att_A_text_T_keywords)
    
    

    #level 3 third signal of article
    #attention article keywords and table title
    att_A_keywords_T_title = attention_method(rep_article_keywords,rep_table_title,rep_article_keywords)
    rep_att_A_keywords_T_title = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_keywords_T_title)
    vector_A_keywords_T_title = tf.keras.layers.Flatten()(rep_att_A_keywords_T_title)
    
    #attention article keywords and table text
    att_A_keywords_T_text = attention_method(rep_article_keywords,rep_table_text,rep_article_keywords)
    rep_att_A_keywords_T_text = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_keywords_T_text)
    vector_A_keywords_T_text = tf.keras.layers.Flatten()(rep_att_A_keywords_T_text)
    
    #attention article keywords and table keywords
    att_A_keywords_T_keywords = attention_method(rep_article_keywords,rep_table_keywords,rep_article_keywords)
    rep_att_A_keywords_T_keywords = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(att_A_keywords_T_keywords)
    vector_A_keywords_T_keywords = tf.keras.layers.Flatten()(rep_att_A_keywords_T_keywords)
    

    #learned vector
    final_vector = tf.keras.layers.Concatenate()([vector_A_title_T_title,vector_A_title_T_text,vector_A_title_T_keywords,vector_A_text_T_title,vector_A_text_T_text,vector_A_text_T_keywords,vector_A_keywords_T_title,vector_A_keywords_T_text,vector_A_keywords_T_keywords])
    
    MLP_input = tf.keras.layers.Dense(256, activation='relu')(final_vector)
    dropout1 = tf.keras.layers.Dropout(0.5, name="dropout1")(MLP_input)
    MLP_hidden = tf.keras.layers.Dense(128, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.5, name="dropout2")(MLP_hidden)
    MLP_output = tf.keras.layers.Dense(1,activation='sigmoid')(dropout2)
    
    model = tf.keras.Model(inputs=[article_title,article_main_passage,article_keywords,table_title,table_main_passage, table_keywords],outputs=MLP_output)
    
    return model

In [ ]:
model = attention_model()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=['accuracy'])

In [ ]:
filepath="attention_model_title_main_passage_keywords_1_1_{epoch:02d}_{val_accuracy:.4f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit([train_article_title, train_article_main_passage, train_article_keywords,train_table_title, train_table_main_passage, train_table_keywords], train_label, 
          epochs=100, 
          batch_size=32, 
          verbose=1,
          validation_data=([validation_article_title, validation_article_main_passage, validation_article_keywords, validation_table_title, validation_table_main_passage, validation_table_keywords], validation_label),
          callbacks=callbacks_list)